In [46]:
from hico.makePosVelQuatCsv import MakePosVelQuatCSV
from hico.HicoL0toL1B import HicoL0toL1b
from hico.auxiliary import CNamespace
from hico.cproc_hico import hico_geo
import os
from IPython.core.display import display, HTML
import pandas as pd
import matplotlib.pyplot as pl
from collections import namedtuple as NT
from datetime import datetime as DT
from numpy import float32
from netCDF4 import Dataset as DS
from numpy import array as nparray
from numpy import ones as npones

In [3]:
%matplotlib inline
display(HTML("<style>.container {width:90%} !important<style>"))

In [13]:
ocvarroot = os.getenv('OCVARROOT')

In [18]:
testdatadir = '/accounts/ekarakoy/ocssw/src/l1bgen_hico/tests/testdata/'
dataBaseName = os.path.commonprefix(os.listdir(testdatadir))
basePath = os.path.join(testdatadir, dataBaseName)
pargs = CNamespace(l0file='%s.bil' % basePath, csvfile='%s.csv' % basePath,
                hdrfile='%s.rhdr' % basePath,
                  boresight=[-0.9957, 0.0268, -0.0128],
                  earthfile=os.path.join(ocvarroot, 'hico', 'finals.data'),
                  lpsfile=os.path.join(ocvarroot, 'modis', 'leapsec.dat'))

In [38]:
def FillNC(root_grp_ptr):
    retGps = NT("returnGroups",  "calGrp, productsGrp, navGrp, slaGrp, periodGrp")
    root_grp_ptr.createDimension('samples', 512)
    root_grp_ptr.createDimension('scan_lines', 2000)
    root_grp_ptr.createDimension('bands', 128)
    root_grp_ptr.instrument = 'HICO'
    root_grp_ptr.institution = 'NASA Goddard Space Flight Center'
    root_grp_ptr.resolution = '100m'
    root_grp_ptr.license = 'http://science.nasa.gov/earth-science/earth-science-data/data-information-policy/'
    root_grp_ptr.naming_authority = 'gov.nasa.gsfc.sci.oceandata'
    root_grp_ptr.date_created = DT.strftime(DT.utcnow(), '%Y-%m-%dT%H:%M:%SZ')
    root_grp_ptr.creator_name = 'NASA/GSFC'
    root_grp_ptr.creator_email = 'data@oceancolor.gsfc.nasa.gov'
    root_grp_ptr.publisher_name = 'NASA/GSFC'
    root_grp_ptr.publisher_url = 'http_oceancolor.gsfc.nasa.gov'
    root_grp_ptr.publisher_email = 'data@oceacolor.gsfc.nasa.gov'
    root_grp_ptr.processing_level = 'L1B'
    nav_grp = root_grp_ptr.createGroup('navigation')
    nav_vars = list()
    nav_vars.append(nav_grp.createVariable('sensor_zenith', 'f4', ('scan_lines', 'samples',)))
    nav_vars.append(nav_grp.createVariable('solar_zenith', 'f4', ('scan_lines', 'samples',)))
    nav_vars.append(nav_grp.createVariable('sensor_azimuth', 'f4', ('scan_lines', 'samples',)))
    nav_vars.append(nav_grp.createVariable('solar_azimuth', 'f4', ('scan_lines', 'samples',)))
    nav_vars.append(nav_grp.createVariable('longitudes', 'f4', ('scan_lines', 'samples',)))
    nav_vars.append(nav_grp.createVariable('latitudes', 'f4', ('scan_lines', 'samples',)))
    for var in nav_vars:
        var.units = 'degrees'
        var.valid_min = -180
        var.valid_max = 180
        var.long_name = var.name.replace('_', ' ').rstrip('s')
    retGps.navGrp = nav_grp
    retGps.productsGrp = root_grp_ptr.createGroup('products')
    lt = retGps.productsGrp.createVariable('Lt', 'u2', ('scan_lines', 'samples', 'bands'))
    lt.scale_factor = float32([1 / 50.0])
    lt.add_offset = float32(0)
    lt.units = "W/m^2/micrometer/sr"
    lt.valid_range = nparray([0, 16384], dtype='u2')
    lt.long_name = "HICO Top of Atmosphere"
    lt.wavelength_units = "nanometers"
    # lt.createVariable('fwhm', 'f4', ('bands',))
    lt.fwhm = npones((128,), dtype='f4') * -1
    # wv = lt.createVariable('wavelengths', 'f4', ('bands',))
    lt.wavelengths = npones((128,), dtype='f4')
    lt.wavelength_units = "nanometers"
    retGps.slaGrp = root_grp_ptr.createGroup('scan_line_attributes')
    retGps.slaGrp.createVariable('scan_quality_flags', 'u1', ('scan_lines', 'samples'))
    # Create metadata group and sub-groups
    meta_grp = root_grp_ptr.createGroup('metadata')
    pl_info_grp = meta_grp.createGroup("FGDC/Identification_Information/Platform_and_Instrument_Identification")
    pl_info_grp.Instrument_Short_Name = "hico"
    prc_lvl_grp = meta_grp.createGroup("FGDC/Identification_Information/Processing_Level")
    prc_lvl_grp.Processing_Level_Identifier = "Level-1B"
    retGps.periodGrp = meta_grp.createGroup("FGDC/Identification_Information/Time_Period_of_Content")
    # fill HICO group
    retGps.calGrp = meta_grp.createGroup("HICO/Calibration")
    return retGps

In [15]:
pargs.earthfile

'/accounts/ekarakoy/ocssw/var/hico/finals.data'

In [5]:
hlc = HicoL0toL1b(pargs)

In [7]:
mpvq = MakePosVelQuatCSV(hlc)

In [21]:
pvqcsv = mpvq.paramsDict['pvqFileName']
earth_orient_file = pargs.earthfile
leap_sec_file = pargs.lpsfile

In [19]:
''.join(pargs.lpsfile)

'/accounts/ekarakoy/ocssw/var/modis/leapsec.dat'

In [20]:
pargs.lpsfile

'/accounts/ekarakoy/ocssw/var/modis/leapsec.dat'

In [36]:
ofile=os.path.basename(pargs.l0file).split('bil')[0] + 'nc'

In [50]:
opath = os.path.join('/accounts/ekarakoy/ocssw/src/l1bgen_hico/tests/testdata/',ofile)
opath

'/accounts/ekarakoy/ocssw/src/l1bgen_hico/tests/testdata/iss.2013033.0202.152429.L0.12577.20130204205948.hico.nc'

In [ ]:
'iss.2013033.0202.152429.L0.12577.20130204205948.hico.h5'

In [8]:
hlc.ConvertRaw2Rad()

In [24]:
hicogeo = hico_geo(pqcsv, earth_orient_file, leap_sec_file, pargs.boresight)

In [51]:
with DS(opath, 'w', format='NETCDF4') as root_grp:
    ncGroups = FillNC(root_grp)
    hlc.WriteRadFile(ncGroups.productsGrp, ncGroups.periodGrp)
    hicogeo.write_geo_nc(ncGroups.navGrp, ncGroups.calGrp)